In [1]:
%matplotlib inline

In [2]:
%%info

In [3]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}


In [4]:
spark


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1748020001704_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Spark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.window import Window

# Tipos de datos
from pyspark.sql.types import (
    StringType, FloatType, IntegerType, DateType, StructType, StructField
)

# Funciones de PySpark
from pyspark.sql.functions import (
    col, lit, lower, trim, regexp_replace, udf
)

# Otros
import unicodedata
from functools import reduce
import re



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark = SparkSession.builder \
    .appName("beer_final_project") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
spark._sc.install_pypi_package("pandas")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt1/yarn/usercache/livy/appcache/application_1748020001704_0001/container_1748020001704_0001_01_000001/tmp/spark-3bebecfa-2d74-4c6f-ae4f-70a5ac19bc6a
    Can't uninstall 'python-dateutil'. No files were found to uninstall.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.23.11 requires python-dateutil<=2.9.0,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.

In [8]:
spark._sc.install_pypi_package("matplotlib", "https://pypi.org/simple")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
spark._sc.install_pypi_package("numpy", "https://pypi.org/simple")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
spark._sc.install_pypi_package("scikit-learn", "https://pypi.org/simple")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
ruta_s3 = "s3a://itam-analytics-sofia/beers/clean/beer_profile_and_ratings.csv"

df = spark.read.option("header", "true") \
               .option("inferSchema", "true") \
               .csv(ruta_s3)

df.printSchema()
df.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- style: string (nullable = true)
 |-- brewery: string (nullable = true)
 |-- beer name (full): string (nullable = true)
 |-- description: string (nullable = true)
 |-- abv: string (nullable = true)
 |-- min ibu: string (nullable = true)
 |-- max ibu: string (nullable = true)
 |-- astringency: string (nullable = true)
 |-- body: string (nullable = true)
 |-- alcohol: string (nullable = true)
 |-- bitter: string (nullable = true)
 |-- sweet: string (nullable = true)
 |-- sour: string (nullable = true)
 |-- salty: string (nullable = true)
 |-- fruits: string (nullable = true)
 |-- hoppy: string (nullable = true)
 |-- spices: string (nullable = true)
 |-- malty: string (nullable = true)
 |-- review_aroma: string (nullable = true)
 |-- review_appearance: string (nullable = true)
 |-- review_palate: double (nullable = true)
 |-- review_taste: double (nullable = true)
 |-- review_overall: double (nullable = true)
 |-- number_of_reviews: double (nul

In [13]:
df.count()
df.columns


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['name', 'style', 'brewery', 'beer name (full)', 'description', 'abv', 'min ibu', 'max ibu', 'astringency', 'body', 'alcohol', 'bitter', 'sweet', 'sour', 'salty', 'fruits', 'hoppy', 'spices', 'malty', 'review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'review_overall', 'number_of_reviews']

In [14]:
# Forma df 

(filas, columnas) = (df.count(), len(df.columns))
print("Shape:", (filas, columnas))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Shape: (3197, 25)

In [15]:
df.describe().show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+-----------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+-----------------+-----------------+-----------------+------------------+
|summary|                name|        style|             brewery|    beer name (full)|         description|                 abv|             min ibu|           max ibu|         astringency|                body|             alcohol|              bitter|             sweet|             sour|               salty|              fruits|             hoppy|              spices|               malty|        review_aroma|   review_appearance|    review_palate|     review_taste|   review_overall| num

In [16]:
# Valores nulos por columna 

from pyspark.sql.functions import col, isnan, when, count

df.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c)
    for c in df.columns
]).show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-------+----------------+-----------+---+-------+-------+-----------+----+-------+------+-----+----+-----+------+-----+------+-----+------------+-----------------+-------------+------------+--------------+-----------------+
|name|style|brewery|beer name (full)|description|abv|min ibu|max ibu|astringency|body|alcohol|bitter|sweet|sour|salty|fruits|hoppy|spices|malty|review_aroma|review_appearance|review_palate|review_taste|review_overall|number_of_reviews|
+----+-----+-------+----------------+-----------+---+-------+-------+-----------+----+-------+------+-----+----+-----+------+-----+------+-----+------------+-----------------+-------------+------------+--------------+-----------------+
|   0|    0|      0|               0|          0|  0|      0|      0|          0|   0|      0|     0|    0|   0|    0|     0|    0|     0|    0|           0|                0|            0|           0|             0|                0|
+----+-----+-------+----------------+-----------+---+---

In [17]:
# Estadísticas generales numéricas 

df.describe().show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+-----------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+-----------------+-----------------+-----------------+------------------+
|summary|                name|        style|             brewery|    beer name (full)|         description|                 abv|             min ibu|           max ibu|         astringency|                body|             alcohol|              bitter|             sweet|             sour|               salty|              fruits|             hoppy|              spices|               malty|        review_aroma|   review_appearance|    review_palate|     review_taste|   review_overall| num

In [18]:
# Estadísticas generales

for c in df.columns:
    df.groupBy(c).count().orderBy("count", ascending=False).show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|         name|count|
+-------------+-----+
|  oktoberfest|   12|
|       porter|   10|
|smoked porter|    9|
|christmas ale|    6|
|          ipa|    5|
+-------------+-----+
only showing top 5 rows

+--------------------+-----+
|               style|count|
+--------------------+-----+
|     lager - adjunct|   45|
|lager - european ...|   43|
|wheat beer - hefe...|   42|
|   stout - irish dry|   42|
|      lambic - fruit|   42|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|             brewery|count|
+--------------------+-----+
|boston beer compa...|   40|
|dogfish head brewery|   31|
|      anheuser-busch|   30|
|three floyds brew...|   29|
|victory brewing c...|   23|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|    beer name (full)|count|
+--------------------+-----+
|great lakes brewi...|    1|
|arcense stoombier...|    1|
|north coast brewi...|    1|
|founders brewing ...|    1|
|b

In [19]:
# Contar valores únicos

df.select("name").distinct().count()
df.select("style").distinct().count()
df.select("brewery").distinct().count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

934

### Tabla de variables del dataset 

| Variable              | Tipo       | Descripción                                                                 |
|-----------------------|------------|-----------------------------------------------------------------------------|
| `name`                | Categórica | Nombre corto de la cerveza                                                 |
| `style`               | Categórica | Estilo de cerveza (e.g., IPA, Lager, Stout)                                |
| `brewery`             | Categórica | Nombre de la cervecería                                                    |
| `beer name (full)`    | Texto      | Nombre completo o extendido de la cerveza                                  |
| `description`         | Texto      | Descripción sensorial de la cerveza                                        |
| `abv`                 | Numérica   | Alcohol by Volume (%), contenido alcohólico                                |
| `min ibu`             | Numérica   | Valor mínimo de International Bitterness Units si se conoce                |
| `max ibu`             | Numérica   | Valor máximo de IBU si se conoce                                           |
| `astringency`         | Numérica   | Sensación de sequedad en boca (0–5)                                        |
| `body`                | Numérica   | Cuerpo o densidad percibida (0–5)                                          |
| `alcohol`             | Numérica   | Percepción del alcohol (0–5)                                               |
| `bitter`              | Numérica   | Intensidad del sabor amargo (0–5)                                          |
| `sweet`               | Numérica   | Intensidad del sabor dulce (0–5)                                           |
| `sour`                | Numérica   | Intensidad del sabor ácido (0–5)                                           |
| `salty`               | Numérica   | Percepción salada (0–5)                                                    |
| `fruits`              | Numérica   | Intensidad de notas frutales (0–5)                                         |
| `hoppy`               | Numérica   | Intensidad de lúpulo (0–5)                                                 |
| `spices`              | Numérica   | Percepción de especias (0–5)                                               |
| `malty`               | Numérica   | Intensidad de maltosidad (0–5)                                             |
| `review_aroma`        | Numérica   | Calificación del aroma por usuarios (0–5)                                  |
| `review_appearance`   | Numérica   | Calificación de apariencia por usuarios (0–5)                              |
| `review_palate`       | Numérica   | Calificación del paladar por usuarios (0–5)                                |
| `review_taste`        | Numérica   | Calificación del sabor por usuarios (0–5)                                  |
| `review_overall`      | Numérica   | Calificación global por usuarios (0–5)                                     |
| `number_of_reviews`   | Entera     | Número total de reseñas registradas                                        |


In [20]:
# Renombrar columnas

df = df.toDF(*[c.strip().lower().replace(" ", "_").replace("(", "").replace(")", "") for c in df.columns])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Seleccionar columnas numéricas 

numerical_cols = [
    'abv', 'min_ibu', 'max_ibu', 'astringency', 'body', 'alcohol', 'bitter',
    'sweet', 'sour', 'salty', 'fruits', 'hoppy', 'spices', 'malty',
    'review_aroma', 'review_appearance', 'review_palate',
    'review_taste', 'review_overall', 'number_of_reviews'
]


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Convertir columnas numéricas que aún están como string

from pyspark.sql.functions import col

for c in numerical_cols:
    df = df.withColumn(c, col(c).cast("float"))

df.select(numerical_cols).printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- abv: float (nullable = true)
 |-- min_ibu: float (nullable = true)
 |-- max_ibu: float (nullable = true)
 |-- astringency: float (nullable = true)
 |-- body: float (nullable = true)
 |-- alcohol: float (nullable = true)
 |-- bitter: float (nullable = true)
 |-- sweet: float (nullable = true)
 |-- sour: float (nullable = true)
 |-- salty: float (nullable = true)
 |-- fruits: float (nullable = true)
 |-- hoppy: float (nullable = true)
 |-- spices: float (nullable = true)
 |-- malty: float (nullable = true)
 |-- review_aroma: float (nullable = true)
 |-- review_appearance: float (nullable = true)
 |-- review_palate: float (nullable = true)
 |-- review_taste: float (nullable = true)
 |-- review_overall: float (nullable = true)
 |-- number_of_reviews: float (nullable = true)

In [23]:
# Matriz de correlación

from pyspark.sql.functions import col

for c in numerical_cols:
    df = df.withColumn(c, col(c).cast("float"))
    
    correlaciones = []
    
for col1 in numerical_cols:
    fila = []
    for col2 in numerical_cols:
        corr = df.stat.corr(col1, col2)
        fila.append(corr)
    correlaciones.append(fila)
    
# Imprimir como matriz
for i, row in enumerate(correlaciones):
    nombre_col = numerical_cols[i]
    fila_redondeada = [f"{v:.2f}" if v is not None else "None" for v in row]
    print(f"{nombre_col:20}: {fila_redondeada}")




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

abv                 : ['1.00', '0.49', '0.54', '-0.14', '0.31', '0.52', '0.11', '0.48', '0.09', '-0.36', '0.28', '-0.02', '0.11', '0.21', '-0.29', '-0.25', '-0.24', '-0.15', '-0.14', '0.17']
min_ibu             : ['0.49', '1.00', '0.86', '-0.04', '0.37', '0.28', '0.54', '0.27', '-0.06', '-0.22', '0.08', '0.40', '-0.07', '0.33', '-0.15', '-0.17', '-0.16', '-0.09', '-0.09', '0.23']
max_ibu             : ['0.54', '0.86', '1.00', '-0.05', '0.35', '0.35', '0.48', '0.32', '-0.02', '-0.21', '0.17', '0.34', '0.03', '0.31', '-0.13', '-0.18', '-0.18', '-0.11', '-0.10', '0.23']
astringency         : ['-0.14', '-0.04', '-0.05', '1.00', '-0.03', '-0.09', '0.11', '0.01', '0.55', '0.16', '0.27', '0.30', '-0.03', '-0.09', '0.14', '-0.08', '-0.13', '-0.11', '-0.09', '-0.04']
body                : ['0.31', '0.37', '0.35', '-0.03', '1.00', '0.24', '0.55', '0.48', '-0.12', '-0.16', '-0.03', '0.07', '0.14', '0.74', '-0.14', '-0.11', '-0.13', '-0.10', '-0.10', '0.07']
alcohol             : ['0.52', '0.28', 

### Selección de columnas relevantes, ajustaremos más adelante con la encuesta

In [24]:
print(df.columns)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['name', 'style', 'brewery', 'beer_name_full', 'description', 'abv', 'min_ibu', 'max_ibu', 'astringency', 'body', 'alcohol', 'bitter', 'sweet', 'sour', 'salty', 'fruits', 'hoppy', 'spices', 'malty', 'review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'review_overall', 'number_of_reviews']

In [26]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col

# 1. Definir columnas numéricas de interés
feature_cols = [
    'abv', 'astringency', 'body', 'alcohol', 'bitter',
    'sweet', 'sour', 'salty', 'fruits', 'hoppy', 'spices', 'malty',
    'review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'review_overall'
]

# 2. Subset del DataFrame y eliminar nulos en esas columnas
df_features = df.select("name", *feature_cols)
df_features_clean = df_features.dropna(subset=feature_cols)

# 3. VectorAssembler (corregido para incluir 'name')
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_vec")

# Seleccionamos 'name' + las columnas sensoriales
df_ensamblado = df_features_clean.select("name", *feature_cols)

# Aplicamos el ensamblador (agrega la columna 'features_vec')
df_ensamblado = assembler.transform(df_ensamblado)


# 4. StandardScaler
scaler = StandardScaler(inputCol="features_vec", outputCol="features_scaled", withMean=True, withStd=True)
scaler_model = scaler.fit(df_ensamblado)
df_scaled = scaler_model.transform(df_ensamblado)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### KNN

In [27]:
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
import numpy as np

# Función para calcular distancia coseno entre dos vectores
def cosine_distance_udf(vec1):
    def distancia(vec2):
        dot = float(vec1.dot(vec2))
        norm1 = float(vec1.norm(2))
        norm2 = float(vec2.norm(2))
        if norm1 == 0.0 or norm2 == 0.0:
            return float("nan")
        return 1 - dot / (norm1 * norm2)
    return udf(distancia, DoubleType())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Por ejemplo: "amber"
nombre_base = "amber"

# Obtener el vector escalado de esa cerveza
vector_base = df_scaled.filter(col("name") == nombre_base).select("features_scaled").first()["features_scaled"]


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Agregar columna de distancia
df_recomendaciones = df_scaled.withColumn("cosine_distance", cosine_distance_udf(vector_base)(col("features_scaled")))

# Ordenar por similitud (excluyendo a sí misma)
df_resultado = df_recomendaciones \
    .filter(col("name") != nombre_base) \
    .select("name", "cosine_distance") \
    .orderBy("cosine_distance") \
    .limit(5)

df_resultado.show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+-------------------+
|name                    |cosine_distance    |
+------------------------+-------------------+
|old thumper             |0.16093461570751988|
|esb                     |0.16933200465095055|
|special amber           |0.18608644763773452|
|sawtooth                |0.19202900992627225|
|bleeding buckeye red ale|0.21679615617313364|
+------------------------+-------------------+

In [30]:
import random

def evaluar_recomendador(df_scaled, vector_col="features_scaled", nombre_col="name", muestras=100, top_k=5):
    nombres = df_scaled.select(nombre_col).rdd.flatMap(lambda x: x).collect()
    nombres_muestra = random.sample(nombres, min(muestras, len(nombres)))

    distancias_promedio = []

    for nombre in nombres_muestra:
        # Vector base
        vec_base = df_scaled.filter(col(nombre_col) == nombre).select(vector_col).first()[0]

        # Calcular distancias
        df_temp = df_scaled.withColumn("cosine_distance", cosine_distance_udf(vec_base)(col(vector_col)))

        # Excluirse a sí misma
        recomendaciones = df_temp.filter(col(nombre_col) != nombre).select("cosine_distance").orderBy("cosine_distance").limit(top_k)

        # Promedio de las k distancias
        promedio = recomendaciones.selectExpr("avg(cosine_distance) as mean").first()["mean"]

        if promedio is not None:
            distancias_promedio.append(promedio)

    return {
        "promedio": np.mean(distancias_promedio),
        "desviacion": np.std(distancias_promedio),
        "n evaluadas": len(distancias_promedio)
    }


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
resultados = evaluar_recomendador(df_scaled, vector_col="features_scaled", nombre_col="name", muestras=100, top_k=5)

print("Distancia promedio entre recomendaciones:", resultados["promedio"])
print("Desviacion estandar:", resultados["desviacion"])
print("Numero de cervezas evaluadas:", resultados["n evaluadas"])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Distancia promedio entre recomendaciones: 0.0975241396349748
Desviacion estandar: 0.05465019283524453
Numero de cervezas evaluadas: 100

### Kmeans

In [32]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import matplotlib.pyplot as plt


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Determinar el número óptimo de clusters (método del codo)

inertias = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(featuresCol="features_scaled", k=k, seed=42)
    model = kmeans.fit(df_scaled)  
    inertia = model.summary.trainingCost
    inertias.append(inertia)

# Gráfica del método del codo
plt.figure(figsize=(8, 5))
plt.plot(k_range, inertias, marker='o')
plt.xlabel("Número de clusters (k)")
plt.ylabel("Inercia")
plt.title("Método del codo para determinar k óptimo")
plt.grid(True)
plt.show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# Entrenar el modelo final con el número óptimo de clusters

from pyspark.ml.clustering import KMeans

kmeans_final = KMeans(featuresCol="features_scaled", k=5, seed=42)
model_final = kmeans_final.fit(df_scaled)
df_clustered = model_final.transform(df_scaled)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# Extraer los vectores como columnas para análisis en pandas (solo para graficar localmente)

from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col

df_plot = df_clustered.withColumn("features_array", vector_to_array("features_scaled"))

# Explota el vector en columnas separadas
for i, c in enumerate(feature_cols):
    df_plot = df_plot.withColumn(c + "_scaled", col("features_array")[i])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
#  Convertir a pandas para análisis visual

df_pandas = df_plot.select(["prediction"] + [c + "_scaled" for c in feature_cols]).toPandas()
df_pandas = df_pandas.rename(columns={"prediction": "cluster"})


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# Agrupar por cluster y graficar 

import matplotlib.pyplot as plt

cluster_means = df_pandas.groupby("cluster").mean()

cluster_means.T.plot(kind="bar", figsize=(14, 6))
plt.title("Perfil sensorial promedio por cluster")
plt.xlabel("Atributo sensorial")
plt.ylabel("Valor estandarizado")
plt.xticks(rotation=45, ha='right')
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# Silhouette Score

from sklearn.metrics import silhouette_score
import numpy as np

X = np.vstack(df_pandas[[c + "_scaled" for c in feature_cols]].values)
labels = df_pandas["cluster"].values

score = silhouette_score(X, labels)
print("Silhouette Score:", score)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Silhouette Score: 0.16495464033019236

In [39]:
df_clustered.groupBy("prediction").count().show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  671|
|         2| 1052|
|         3|  390|
|         4|  647|
|         0|  317|
+----------+-----+

In [40]:
from pyspark.sql.functions import avg
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col

df_array = df_clustered.withColumn("features_array", vector_to_array("features_scaled"))

for i, c in enumerate(feature_cols):
    df_array = df_array.withColumn(c + "_scaled", col("features_array")[i])
    

df_avg = df_array.groupBy("prediction").agg(
    *[avg(c + "_scaled").alias(c) for c in feature_cols]
).orderBy("prediction")


pandas_avg = df_avg.toPandas().set_index("prediction").T


pandas_avg.plot(kind="bar", figsize=(14, 6))
plt.title("Perfil sensorial promedio por cluster")
plt.xlabel("Atributo sensorial")
plt.ylabel("Valor estandarizado")
plt.xticks(rotation=45, ha="right")
plt.legend(title="Cluster")
plt.tight_layout()
plt.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Modelo Híbrido: Kmeans + KNN 

- Input: nombre de cerveza (nombre_cerveza).

- Busca: si hay match parcial en el nombre en df.

- Extrae: su índice (index_ref), su nombre y su cluster_id.

- Filtra: todas las cervezas en ese mismo cluster. (Kmeans)

- Reentrena un KNN: solo con las cervezas dentro del cluster.

- Encuentra las más parecidas: calcula distancia coseno desde la cerveza base.

- Devuelve: las n cervezas más similares (exceptuando la base).

In [42]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col, lit, lower, array
from pyspark.sql import functions as F
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
# Simulación del input
nombre_cerveza = "guinness"


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
# Búsqueda parcial
base = df_clustered.filter(F.lower(F.col("name")).contains(nombre_cerveza.lower()))
base.show(truncate=False)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------+---+-----------+-----+-------+------+-----+----+-----+------+-----+------+-----+------------+-----------------+-------------+------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|name                                 |abv|astringency|body |alcohol|bitter|sweet|sour|salty|fruits|hoppy|spices|malty|review_aroma|review_appearance|review_palate|review_taste|review_overall|features_vec                                                                                   

In [52]:
# Extraer nombre y cluster_id

row = base.first()

if row:
    nombre_ref = row["name"]
    cluster_id = row["prediction"]
    print(f"Cerveza base: {nombre_ref}")
    print(f"Cluster sensorial asignado: {cluster_id}")
else:
    print("No se encontró la cerveza.")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cerveza base: guinness black lager
Cluster sensorial asignado: 3

In [53]:
# Filtrar el cluster correspondiente

df_cluster = df_clustered.filter(F.col("prediction") == cluster_id)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
# Convertir a pandas para usar KNN 

from pyspark.ml.functions import vector_to_array

df_cluster_array = df_cluster.withColumn("features_array", vector_to_array("features_scaled"))
df_cluster_pd = df_cluster_array.select("name", "features_array").toPandas()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
# Aplicar KNN en numpy

from sklearn.neighbors import NearestNeighbors
import numpy as np

X = np.vstack(df_cluster_pd["features_array"])
names = df_cluster_pd["name"].values

idx_ref = np.where(names == nombre_ref)[0][0]
top_n = min(6, len(names))  # 5 recomendaciones + 1 base

knn = NearestNeighbors(n_neighbors=top_n, metric="cosine")
knn.fit(X)

distances, indices = knn.kneighbors([X[idx_ref]])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
# Mostrar resultados 

print(f"\n Recomendaciones para: {nombre_ref}\n")
for i, idx in enumerate(indices[0][1:], 1):
    print(f"{i}. {names[idx]} – Distancia: {distances[0][i]:.4f}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 Recomendaciones para: guinness black lager

1. orion mugi shokunin ? Distancia: 0.0197
2. dark wheat ? Distancia: 0.0335
3. kirin classic lager ? Distancia: 0.0389
4. san miguel selecta ? Distancia: 0.0433
5. mississippi mud black & tan ? Distancia: 0.0463

In [59]:
# Encapsular todo en una función

def recomendar_hibrido_spark(nombre_cerveza, df_clustered, top_n=5):
    from pyspark.ml.functions import vector_to_array
    from sklearn.neighbors import NearestNeighbors
    import numpy as np

    # Búsqueda
    base = df_clustered.filter(F.lower(F.col("name")).contains(nombre_cerveza.lower()))
    row = base.first()
    if not row:
        print("No se encontró la cerveza.")
        return

    nombre_ref = row["name"]
    cluster_id = row["prediction"]

    print(f"\n Cerveza base: {nombre_ref}")
    print(f" Cluster sensorial: {cluster_id}")

    # Subconjunto
    df_cluster = df_clustered.filter(F.col("prediction") == cluster_id)
    df_array = df_cluster.withColumn("features_array", vector_to_array("features_scaled"))
    df_pd = df_array.select("name", "features_array").toPandas()

    # KNN
    X = np.vstack(df_pd["features_array"])
    names = df_pd["name"].values
    idx_ref = np.where(names == nombre_ref)[0][0]

    knn = NearestNeighbors(n_neighbors=min(top_n+1, len(names)), metric="cosine")
    knn.fit(X)
    distances, indices = knn.kneighbors([X[idx_ref]])

    print("\n Recomendaciones:")
    for i, idx in enumerate(indices[0][1:], 1):
        print(f"{i}. {names[idx]} – Distancia: {distances[0][i]:.4f}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
recomendar_hibrido_spark("guinness", df_clustered, top_n=5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 Cerveza base: guinness black lager
 Cluster sensorial: 3

 Recomendaciones:
1. orion mugi shokunin ? Distancia: 0.0197
2. dark wheat ? Distancia: 0.0335
3. kirin classic lager ? Distancia: 0.0389
4. san miguel selecta ? Distancia: 0.0433
5. mississippi mud black & tan ? Distancia: 0.0463